In [ ]:
# Installing all libraries for scraping data
!pip install requests
!pip install beautifulsoup4
!pip install requests-html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.5 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=295cf5e7b90cbfb7a438341d9a129bfd1372e2b5f2abad0ad6497a30b1e24f37
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [ ]:
from requests_html import HTMLSession
import re
import pandas as pd


# establish a session
session = HTMLSession()

In [ ]:
# After observing url_pattern for searching in MBFC site, I have devised this function for automating collection of search response
def get_search_url(url):
  root_url = "https://mediabiasfactcheck.com/?s="
  sum_url = root_url + url
  resp = session.get(sum_url)
  return resp

After observing patterns of urls containing bias and search result of them, I devised this function to match exact pattern of links . Links containing bias and credibility , if being searched , get to the page containing detailed report frequently.

In [ ]:
def get_bias_search_url_tool(url):
  response = get_search_url(url)
  bias_links = response.html.find("a", containing = "bias")
  pattern = r"<Element 'a' href='https://mediabiasfactcheck.com/([^/]+)/' title='([^']+) Bias and Credibility' rel=\('bookmark',\)>"
  pattern2 = r"<Element 'a' href='https://mediabiasfactcheck.com/([^/]+)/' title='([^']+)(?: Bias| Biased)' rel=\('bookmark',\)>"

  return_final = []

  for x in bias_links:
    return_to_be =[]
    biasdict = None
    url_bias_dict = None
    biascredict = None
    url_biascred_dict = None

    tex = str(x)
    match1 = re.search(pattern,tex)
    match2 = re.search(pattern2,tex)
    if match2:
      # Access the captured groups using match.group(1) and match.group(2)
      anything = match2.group(1)
      url_bias = 'https://mediabiasfactcheck.com/'+anything+'/'
      print(url_bias)
      anything_title = match2.group(2)
      print(f"Matched 'anything': {anything}")
      print(f"Matched 'Anything' (title): {anything_title}")
      biasdict = {"bias": anything_title}
      url_bias_dict = {"search_bias_url":url_bias}
    elif match1:
      anything2 = match1.group(1)
      url_biascred = 'https://mediabiasfactcheck.com/'+anything2+'/'
      print(url_biascred)
      anything_title2 = match1.group(2)
      print(f"Matched 'anything': {anything2}")
      print(f"Matched 'Anything' (title): {anything_title2}")
      biascredict = {"biascred": anything2}
      url_biascred_dict = {"search_biascred_url":url_biascred}

    return_to_be.append(url_biascred_dict)
    return_to_be.append(biascredict)
    return_to_be.append(biasdict)
    return_to_be.append(url_bias_dict)

    for item in return_to_be:
      if item is not None:
        return_final.append(return_to_be)
        break

  result = []
  for it in return_final:
    list_be = it
    for tm in list_be:
      if tm is not None:
        result.append(tm)

  return result

In [ ]:
# Functions to get bias_credibility urls and bias_urls separately and conveniently.
# These functions use output result of 'get_bias_search_tools' function as input, which is supposed to be a dictionary
def get_bias_credibility_url(result_dict_tools):
  for rs in result_dict_tools:
    if 'search_biascred_url' in rs.keys():
      url = rs['search_biascred_url']
      if url is not None:
        print("search_biascred_url:"+ url)
        break
    else:
      url = None

  return url

def get_bias_url(result_dict_tools):
  for rs in result_dict_tools:
    if 'search_bias_url' in rs.keys():
      url = rs['search_bias_url']
      if url is not None:
        print("search_bias_url:"+ url)
        break
    else:
      url = None

  return url


In [ ]:
# This function extracts Bias and Factual Reporting from detailed report page using string matching patterns
# First, texts are extracted from html of response for link.
# Then string matching patterns are searched in loaded texts.
def search_for_detailed_report(response):
  pattern3 = r"Bias Rating:([^/]+)"
  pattern4 = r"Factual Reporting:([^/]+)"
  resptext = response.html.text
  match3 = re.search(pattern3,resptext)
  match4 = re.search(pattern4,resptext)
  if (match3 is not None) & (match4 is not None):
    list_match3 = match3.group(1).split("\n")
    list_match4 = match4.group(1).split("\n")
    flag_d = True
  else:
    list_match3 = [None]
    list_match4 = [None]
    flag_d = False
  return {"bias_rating":list_match3[0],"fact_report":list_match4[0],"report_flag": flag_d}


In [ ]:
# Extracting 'href' value because url values are used as values of 'href' in html.
def collect_href_url(html_text):
  href_pattern = r"href='([^']+)'"
  # Use re.search to find the first match in the HTML text
  match = re.search(href_pattern, str(html_text))
  if match:
    href_value = match.group(1)
  else:
    href_value = None
  return href_value

By observing, these three points are noticed in MBFC site:


*   link containing bias information of a media/news site frequently contains 'read more' link
*   This 'read more' link often turns to the page containing links with similar bias and fact information
*   Clicking one of those links often turns to page containing detailed report about bias and fact check , which is our target
*   In some cases, 'read more' link itself turns to page containing detailed bias and fact information





In [ ]:
# Devising a function to search in read_more
def search_in_read_more(bias_link):
    url_tree1 = bias_link
    resp_tree1 = session.get(url_tree1)
    rm_link = resp_tree1.html.find("a", containing="read more")

    if rm_link:
        url_read_more = collect_href_url(rm_link)
        resp_rm = session.get(url_read_more)
        detailed_dict = search_for_detailed_report(resp_rm)

        if detailed_dict["report_flag"] is False:
            links = resp_rm.html.find("a")
            link_pattern = r"<Element 'a' href='https://mediabiasfactcheck.com/([^/]+)/'>"
            matches_link = [re.search(link_pattern, str(l)).group(1) for l in links if re.search(link_pattern, str(l))]

            remove_words = ['left-vs-right-bias-how-we-rate-the-bias-of-media-sources', 'support-media-bias-fact-check', 'filtered-search', 'presidential-election-2024-candidates-bias-ratings', 'fair-use-policy', 'funding', 'about', 'contact']
            matches_link = [item for item in matches_link if item not in remove_words]

            if matches_link:
                item_str = str(matches_link[0])
                url_tree2 = 'https://mediabiasfactcheck.com/' + item_str + '/'
                resp_tree2 = session.get(url_tree2)
                detail_report_dict = search_for_detailed_report(resp_tree2)
            else:
                detail_report_dict = None
        else:
            detail_report_dict = detailed_dict

        return detail_report_dict

    else:
      links = resp_tree1.html.find("a")
      link_pattern = r"<Element 'a' href='https://mediabiasfactcheck.com/([^/]+)/'>"
      matches_link = [re.search(link_pattern, str(l)).group(1) for l in links if re.search(link_pattern, str(l))]
      remove_words = ['left-vs-right-bias-how-we-rate-the-bias-of-media-sources', 'support-media-bias-fact-check', 'filtered-search', 'presidential-election-2024-candidates-bias-ratings', 'fair-use-policy', 'funding', 'about', 'contact']
      matches_link = [item for item in matches_link if item not in remove_words]
      if matches_link:
        item_str = str(matches_link[0])
        url_tree2 = 'https://mediabiasfactcheck.com/' + item_str + '/'
        resp_tree2 = session.get(url_tree2)
        detail_report_dict = search_for_detailed_report(resp_tree2)
      else:
        detail_report_dict = None

    return detail_report_dict


In this stage , most of the helper functions are prepared . Now our target is finding detailed report.
From observation:


*   links containing bias and credibility often directly leads to detailed report . So these are the first queries to search
*   Then , check if these links lead to the page containing 'read more' link. If yes, the 'read more' link is searched.
*   if detailed information is not found with above steps , then url containing bias is searched directly
*   Then the 'read more' link is searched if it leads to the page containing detailed report






In [ ]:
def find_detailed_report(url):
    get_tools = get_bias_search_url_tool(url)

    url_bias = get_bias_url(get_tools)
    url_biascred = get_bias_credibility_url(get_tools)

    if url_biascred is not None:
        sep3 = session.get(url_biascred)
        factcheck_report_trial1 = search_for_detailed_report(sep3)
        if factcheck_report_trial1["report_flag"]:
            print(factcheck_report_trial1)
            print("factcheck successful at trial1")
            return factcheck_report_trial1

        factcheck_report_trial2 = search_in_read_more(url_biascred)
        if factcheck_report_trial2:
            print("factcheck successful at trial2")
            return factcheck_report_trial2

    if url_bias is not None:
        sep4 = session.get(url_bias)
        factcheck_report_trial3 = search_for_detailed_report(sep4)
        if factcheck_report_trial3["report_flag"]:
            print("factcheck successful at trial3")
            return factcheck_report_trial3

        factcheck_report_trial4 = search_in_read_more(url_bias)
        if factcheck_report_trial4["report_flag"]:
            print("factcheck successful at trial4")
            return factcheck_report_trial4

    return None


Now, devising a function to get bias and fact information extracted from a media link. If any error occurs and information collecting gets failed, these erronous urls must be stored to be handled further.

In [ ]:
def get_factreport_extracted(url):
  rt = None
  error_url = None
  if url is not None:
      try:
        rt = find_detailed_report(url)
        print(rt)
      except:
        print("error occured")
        error_url = url

  return{ 'fact_dict': rt, 'error_url': url}

To handle erronous links, after observing the patterns of errors , I get that searching some links are producing multiple bias and biascred urls getting mixed up. I need to identify them and store necessary url into separate lists.


---

For above reason, I devised a function which takes erronuous urls and store representative urls divided into two lists:


*   'biascred_list': which contains bias-credibility urls
*   'bias_for_read_more': which contains urls with only type of bias



In [ ]:
def get_error_handled(errors):
  res_bias_search = [] # Empty list for storing bias search result
  for url in errors:
    res = get_bias_search_url_tool(url)
    res_bias_search.append(res)
  cnt = 0
  search_link_list = []
  bias_for_read_more = []
  biascred_list = []
  index_list = []
  for item in res_bias_search:  # each result stored in this list is supposed to be list of dictionaries.
    print(cnt)
    if len(item) > 0:
      search_link = None # Forming url for further search
      length = len(item)
      item_dict = item[length - 1]
      if 'search_bias_url' in item_dict.keys():
        search_link = item_dict['search_bias_url']
        search_dict ={'err_list_index': cnt , 'search_link': search_link} # here, search_link is only bias url
        bias_for_read_more.append(search_dict) # here too
      elif 'bias' in item_dict.keys():
        bias_val = item_dict['bias'] # here , we get only type of bias [ As example , Left-Center]
        bias_val = bias_val.replace('-','')
        bias_val = bias_val.replace(' ','')
        bias_val = bias_val.lower()
        search_link = 'https://mediabiasfactcheck.com/'+bias_val+'/'  # So forming search link containing only bias
        search_dict ={'err_list_index': cnt , 'search_link': search_link}
        bias_for_read_more.append(search_dict)
      elif 'search_biascred_url' in item_dict.keys():
        search_link = item_dict['search_biascred_url'] # Here we get only bias-credibility url
        search_dict ={'err_list_index': cnt , 'search_link': search_link} # forming search link containing bias-credibility
        biascred_list.append(search_dict)
      elif 'biascred' in item_dict.keys():
        biascred = item_dict['biascred'] # Here , we get only bias credibility word
        search_link = 'https://mediabiasfactcheck.com/'+biascred+'/'
        search_dict ={'err_list_index': cnt , 'search_link': search_link}# forming search link containing bias-credibility
        biascred_list.append(search_dict)
      else:
        print(item_dict.keys())
    cnt = cnt +1

    return {'bias_err_link': bias_for_read_more,'biascred_err_link': biascred_list} # dictionary containing two lists







Now, I need helper functions to handle these erronous links by executing their representative links with bias or representative links with bias and credibility in same way.


> The search-links are searched for detailed report


> 'find_detailed_report' function provide a dictionary which contains :
    
*   bias_rating : value of this key is type of bias for the url
*   factual_report: value of this key is measure of fact for the url
*   report_flag : If value is True, fact check is successful . If value is False, fact check is not successful.

This output dictionary is called here as 'fact_dict'



> The final output structure is a list of dictionaries containing:


*  Dictionary of report called 'Report_dict'
*  Dictionary of url














In [ ]:
def bias_error_links_handled(bias_err_links , errors):
  df_list = []
  cnt = 0
  for item in bias_err_links:
    print(cnt)
    try:
      rt = find_detailed_report(item['search_link'])
      print(rt)
      ind = item['err_list_index']
      url = errors[ind]
      report_dict = {'fact_dict': rt , 'error_url': url}
      df_dict = {'url':url,'report_dict':report_dict}
      df_list.append(df_dict)
    except:
      print('error index:', cnt)
    cnt = cnt + 1

  return df_list

In [ ]:
def biascred_err_links_handled(biascred_err_links,errors):
  df_list2 = []
  cnt = 0
  for item in biascred_err_links:
    print(cnt)
    try:
      resp = session.get(item['search_link'])
      rt = search_for_detailed_report(resp)
      print(rt)
      ind = item['err_list_index']
      url = errors[ind]
      report_dict = {'fact_dict': rt , 'error_url': url}
      df_dict = {'url':url,'report_dict':report_dict}
      df_list2.append(df_dict)
    except:
      print('error index:', cnt)
    cnt = cnt + 1
  return df_list2

Now I am devising a function which will:



*   Take input of a primary dataframe containing only urls.
*   Gets 'get_factreport_extracted' function applied to each url in column of dataframe for generating column named 'report_dict'
*   Generates list of erronous urls for which fact information is none.
*   Gets erronous urls handled by error handler functions
*   Now replaces value of 'report_dict' for each erronous url with correct value of 'report_dict' in dictionary of dataframe
*   Converting dictionary of clean data into clean dataframe


In [ ]:
def root_error_handled_step_1(primary_df):
  df = primary_df
  df['report_dict'] = df['url'].apply(get_factreport_extracted)
  urls_in_dataframe = list(df['url'])
  report_dict_list = list(df['report_dict'])
  errors = []

  for ind in range(len(report_dict_list)):
    repdict = report_dict_list[ind]
    if repdict['fact_dict'] is None:
      err = urls_in_dataframe[ind]
      errors.append(err)

  errors_dict = get_error_handled(errors)
  bias_err_links = errors_dict['bias_err_link']
  biascred_err_links = errors_dict['biascred_err_link']
  df_list =  bias_error_links_handled(bias_err_links , errors)
  df_list2 = biascred_err_links_handled(biascred_err_links,errors)

  for ind in range(len(urls_in_dataframe)):
    for item in df_list:
      if urls_in_dataframe[ind] == item['url']:
        report_dict_list[ind] = item['report_dict']
        break
    for item2 in df_list2:
      if urls_in_dataframe[ind] == item2['url']:
        report_dict_list[ind] = item2['report_dict']
        break

  df_clean_dict = {'url': urls_in_dataframe,'report_dict': report_dict_list}
  df_clean = pd.DataFrame(df_clean_dict)

  return df_clean

Now, for preparing final dataset , we need to extract
 data dictionary for each url in each row of clean dataframe.

The CSV dataset should contain  columns:


*   url
*   fact
*   bias
*   Normalized_url

The function 'extract_prep_row_for_csv(row)' takes a row of dataframe as input and store extracted information url, fact and bias in a dictionary for converting into CSV file.





In [ ]:
def extract_prep_row_for_csv(row):
  url = row['url']
  report = row['report_dict']
  biasdict = report['fact_dict']
  bias = None
  fact = None
  if biasdict is not None:
    bias = biasdict['bias_rating']
  factdict = report['fact_dict']
  if factdict is not None:
    fact = factdict['fact_report']
  repdict = {'url': url,'bias' : bias ,'fact' : fact}
  return repdict


The primary dataframe should contain all urls loaded from our text file and cleaned primarily.

In [ ]:
def get_primary_df(filepath):
  news_url = []
  with open(filepath,'r+') as file:
    for line in file:
      news_url.append(line)

  urls = []
  for item in news_url:
    if item == '\n':
      news_url.remove(item)
    else:
      it_st = str(item)
      it_st = it_st.replace('\n','')
      it_st = it_st.replace('\t','')
      it_st = it_st.replace(' ','')
      urls.append(it_st)

  clean_url_dict = {'url': urls}
  pm_df = pd.DataFrame(clean_url_dict)

  return pm_df


Now , the second step of dataset preparation involves:


*   Applying all the functions for getting clean dataframe from primary dataframe
*   Droping any row which contains empty value



In [ ]:
def get_dataset_prepared(filepath):
  pmdf = get_primary_df(filepath)
  clean_df = root_error_handled_step_1(pmdf)
  status_count = 0
  csv_list = []
  for index,row in clean_df.iterrows():
    print(status_count)
    report_dict = row['report_dict']
    fact_dict_val = report_dict['fact_dict']
    if fact_dict_val is None:
      clean_df.drop(index)
    else:
      repdict = extract_prep_row_for_csv(row)
      csv_list.append(repdict)
    status_count = status_count + 1

  prepared_df = pd.DataFrame(csv_list)

  return prepared_df


*   Normalized url = Root url
*   For extracting root url, I need to use string matching pattern here.
*   Matched pattern is extracted result


In [ ]:
def get_normalized_url(url):
  normalized_url_pattern1 = r'^(https?://[^/]+/)'
  normalized_url_pattern2 = r'^(https?://[^/]+)'

  match1 = re.match(normalized_url_pattern1, url)
  match2 = re.match(normalized_url_pattern2, url)

  if match1:
    normalized_url = match1.group(1)
    print("Normalized URL:", normalized_url)
  elif match2:
    normalized_url = match2.group(1)
    print("Normalized URL:", normalized_url)
  else:
    normalized_url = None
    print("Normalized URL not found in the provided URL.")

  return normalized_url

Now getting clean dataframe from the text file by using 'get_dataset_prepared' function

In [ ]:
prep_df = get_dataset_prepared('/content/sample_data/cleaned_news_urls.txt')

Streaming output truncated to the last 5000 lines.
https://mediabiasfactcheck.com/investing-com/
Matched 'anything': investing-com
Matched 'Anything' (title): Investing.com –
https://mediabiasfactcheck.com/electoral-vote-com/
Matched 'anything': electoral-vote-com
Matched 'Anything' (title): Electoral-Vote.com –
https://mediabiasfactcheck.com/law-com/
Matched 'anything': law-com
Matched 'Anything' (title): Law.com –
https://mediabiasfactcheck.com/24ur-com/
Matched 'anything': 24ur-com
Matched 'Anything' (title): 24ur.com –
https://mediabiasfactcheck.com/boston-com/
Matched 'anything': boston-com
Matched 'Anything' (title): Boston.com –
https://mediabiasfactcheck.com/sohu-com/
Matched 'anything': sohu-com
Matched 'Anything' (title): Sohu.com –
https://mediabiasfactcheck.com/right-center/
Matched 'anything': right-center
Matched 'Anything' (title): Right-Center
https://mediabiasfactcheck.com/leftcenter/
Matched 'anything': leftcenter
Matched 'Anything' (title): Left-Center
https://mediab

Removing null rows for fact column

In [ ]:
prep_df_clean = prep_df[pd.notna(prep_df['fact'])]

Now, preparing new column 'Normalized_url' by applying 'get_normalized_url' function in all rows of 'url' column

In [ ]:
prep_df_clean['Normalized_url'] = prep_df_clean['url'].apply(get_normalized_url)

Normalized URL: https://www.bbc.com/
Normalized URL: https://www.msn.com/
Normalized URL: https://www.cnn.com
Normalized URL: https://edition.cnn.com/
Normalized URL: https://www.nytimes.com/
Normalized URL: https://www.dailymail.co.uk/
Normalized URL: https://news.google.com/
Normalized URL: https://www.theguardian.com/
Normalized URL: https://www.foxnews.com/
Normalized URL: https://news.yahoo.com/
Normalized URL: https://finance.yahoo.com/
Normalized URL: https://timesofindia.indiatimes.com/
Normalized URL: https://www.indiatimes.com/
Normalized URL: https://people.com/
Normalized URL: https://www.hindustantimes.com/
Normalized URL: http://the-sun.co/
Normalized URL: https://www.usatoday.com/
Normalized URL: https://nypost.com/
Normalized URL: https://www.washingtonpost.com/
Normalized URL: https://www.news18.com/
Normalized URL: https://www.ndtv.com/
Normalized URL: https://www.cnbc.com/
Normalized URL: https://www.forbes.com/
Normalized URL: https://www.mirror.co.uk/
Normalized UR

<ipython-input-36-729f5a95e669>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_df_clean['Normalized_url'] = prep_df_clean['url'].apply(get_normalized_url)


In [ ]:
len(prep_df_clean)

992

Finally converting this clean dataframe to csv file

In [ ]:
csv_file_path = '/content/sample_data/prep_csv.csv'
prep_df_clean.to_csv(csv_file_path, index=False)